In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=096e0b75c243142038276778a0d213bbaea0f1ac26015fd444e47481cf9862ff
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers


In [2]:
!nvidia-smi

Thu Mar  2 10:56:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from datasets import load_dataset
data_full = load_dataset("csebuetnlp/BanglaParaphrase")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset indic_paraphrase downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___indic_paraphrase/bn/1.0.0/07c4922f871f96b07ad70543296483a0316f01890648accf9cf81c08b1934cce. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset = data_full['train']

In [5]:
dataset[0]

{'source': 'বিমানটি যখন মাটিতে নামার জন্য এয়ারপোর্টের কাছাকাছি আসছে, তখন ল্যান্ডিং গিয়ারের খোপের ঢাকনাটি খুলে যায়।',
 'target': 'বিমানটা এয়ারপোর্টের দিকে ল্যান্ড করার সময়, ল্যান্ডিং গিয়ার প্যানেলের ঢাকনাটা খুলে দেয়া হয়।'}

In [6]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm  

train_samples = []
for data in tqdm(dataset):
    train_samples.append(InputExample(
        texts=[data['source'], data['target']]
    ))

  0%|          | 0/419967 [00:00<?, ?it/s]

In [7]:
from sentence_transformers import datasets

# batch_size = 32
batch_size = 10

loader = datasets.NoDuplicatesDataLoader(
    train_samples, batch_size=batch_size)

In [8]:
from sentence_transformers import SentenceTransformer



model = SentenceTransformer('afschowdhury/s-xlmr-bn')

model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [9]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(model)

In [10]:
import os
out_dir = "st_model_v1.2"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [11]:
epochs = 2
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    optimizer_params={'lr': 2e-5, 'eps': 1e-6},
    save_best_model=True,
    checkpoint_path = out_dir+'/checkpoint',
    checkpoint_save_steps = 1500,
    checkpoint_save_total_limit = 2,
    output_path= out_dir,
    show_progress_bar=True
)  

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/41996 [00:00<?, ?it/s]

Iteration:   0%|          | 0/41996 [00:00<?, ?it/s]